In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch
from torch.utils.data import TensorDataset, DataLoader
import os

In [31]:
# path = os.path.join("..","..","data","YN_final_df.csv")
batch_size = 4096
GPU_available = False 
embedding_dim = 70
epoch = 100
lr = 1e-2

In [32]:
ToTensor = torch.LongTensor
num_user = 0
if GPU_available:
    ToTensor = torch.cuda.LongTensor

In [13]:
path = os.path.join("..","..","data","YN_final_df.csv")

import pandas as pd 
df = pd.read_csv(path) 
df.head()

/Users/apple/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,locationId,place.name,category,rating,createdDate,is_fch,photonum,is_local,rated_count,average_photonum,average_rating,user_mean_rating,user_reviewcount,userID,category_l,land.addr
0,788542,마르코 폴로,EAT,5.0,20180314,0,0,1,20,0.05,3.95,4.363636,11,14256,1,서울 강남구 삼성동 159-1 트레이드타워 52층
1,788542,마르코 폴로,EAT,4.0,20171207,0,0,1,20,0.05,3.95,4.000000,6,722,1,서울 강남구 삼성동 159-1 트레이드타워 52층
2,788542,마르코 폴로,EAT,5.0,20161110,0,0,0,20,0.05,3.95,5.000000,1,30372,1,서울 강남구 삼성동 159-1 트레이드타워 52층
3,788542,마르코 폴로,EAT,3.0,20160611,0,0,0,20,0.05,3.95,4.250000,4,2116,1,서울 강남구 삼성동 159-1 트레이드타워 52층
4,788542,마르코 폴로,EAT,4.0,20151204,0,0,1,20,0.05,3.95,4.095238,42,3208,1,서울 강남구 삼성동 159-1 트레이드타워 52층


In [47]:
columns = ['locationId','createdDate',
       'is_fch', 'photonum', 'is_local', 'rated_count', 'average_photonum',
       'average_rating', 'user_mean_rating', 'user_reviewcount', 'userID',
       'category_l']

group = []
label = []
for col in columns:
    group.append([i for i in df[col].values])

label.append([int(i) for i in df['rating'].values])

In [20]:
def get_data(column, df) :
    
    vocab = {}
    num = 0

    for i in df[column]: # np.hstack([train[column], test[column]]): 
        if vocab.get(i) != None:
            continue

        vocab[i] = num
        num += 1

    encoded = [vocab[i] for i in df[column]]
    return encoded, num, vocab

encoded_locationId, num_locationId, vocab_locationId = get_data('locationId', df) 
encoded_createdDate,  num_createdDate, vocab_createdDate = get_data('createdDate', df) 
encoded_is_fchm,  num_is_fch, vocab_is_fch = get_data('is_fch', df) 
encoded_photonum,  num_photonum, vocab_photonum = get_data('photonum', df) 
encoded_is_local,  num_is_local, vocab_is_local = get_data('is_local', df) 
encoded_rated_count,  num_rated_count, vocab_rated_count = get_data('rated_count', df) 
encoded_average_photonum,  num_average_photonum, vocab_average_photonum = get_data('average_photonum', df) 
encoded_average_rating,  num_average_rating, vocab_average_rating = get_data('average_rating', df) 
encoded_users_mean_rating, num_users_mean_rating, vocab_users_mean_rating = get_data('user_mean_rating', df) 
encoded_user_reviewcount,  num_user_reviewcount, vocab_user_reviewcount = get_data('user_reviewcount', df) 
encoded_user_reviewcount,  num_user_reviewcount, vocab_user_reviewcount = get_data('user_reviewcount', df) 
encoded_userID, num_userID, vocab_userID = get_data('userID', df) 
encoded_category_l, num_category_l, vocab_category_l = get_data('category_l', df) 


In [21]:
feat_nu_dict = {}
feat_nu_dict['locationId'] = num_locationId
feat_nu_dict['createdDate'] = num_createdDate
feat_nu_dict['is_fch'] = num_is_fch
feat_nu_dict['photonum'] = num_photonum
feat_nu_dict['is_local'] = num_is_local
feat_nu_dict['rated_count'] = num_rated_count
feat_nu_dict['average_photonum'] = num_average_photonum
feat_nu_dict['average_rating'] = num_average_rating
feat_nu_dict['user_mean_rating'] = num_users_mean_rating
feat_nu_dict['user_reviewcount'] = num_user_reviewcount
feat_nu_dict['userID'] = num_userID
feat_nu_dict['category_l'] = num_category_l

feat_nu_dict 

{'locationId': 16249,
 'createdDate': 4346,
 'is_fch': 3,
 'photonum': 52,
 'is_local': 2,
 'rated_count': 312,
 'average_photonum': 1954,
 'average_rating': 4401,
 'user_mean_rating': 35947,
 'user_reviewcount': 176,
 'userID': 105141,
 'category_l': 2}

In [61]:
groups = []
groups.append((group[0], feat_nu_dict['locationId'], 'locationId'))
groups.append((group[1], feat_nu_dict['createdDate'], 'createdDate'))
groups.append((group[2], feat_nu_dict['is_fch'], 'is_fch'))
groups.append((group[3], feat_nu_dict['photonum'], 'photonum'))
groups.append((group[4], feat_nu_dict['is_local'], 'is_local'))
groups.append((group[5], feat_nu_dict['rated_count'], 'rated_count'))
groups.append((group[6], feat_nu_dict['average_photonum'], 'average_photonum'))
groups.append((group[7], feat_nu_dict['user_mean_rating'], 'user_mean_rating'))
groups.append((group[8], feat_nu_dict['user_reviewcount'], 'user_reviewcount'))
groups.append((group[9], feat_nu_dict['userID'], 'userID'))
groups.append((group[10], feat_nu_dict['category_l'], 'category_l'))
len(groups)

11

In [ ]:
ToTensor = torch.LongTensor
GPU = GPU_available
if GPU:
    ToTensor = torch.cuda.LongTensor
    device = torch.device('cuda')

col_embed = nn.Embedding(num_user+1,embedding_dim)
prediction = nn.Linear(embedding_dim,1)
attention= Attention(embedding_dim)

    
p = nn.Parameter(torch.rand(1,embedding_dim))
# first = nn.Embedding(num_user,1) 
# gen_bias = nn.Parameter(torch.rand(1)) 

In [ ]:
def FirstOrder(group):
    return torch.sum(nn.Embedding(group,1), dim=1).reshape(-1)

In [ ]:
def AttentiveSecond(group):
    gr_len = len(x[0])
    gr_embed = []
    a = torch.FloatTensor()
    if GPU_available:
        a = torch.cuda.FloatTensor()

    for x in group:
        gr_embed.append(nn.Embedding(input_dim = x[1], output_dim = embedding_dim, name = x[2])(ToTensor(x[0])))

    for gr in gr_embed:
        b = torch.FloatTensor()
        if GPU_available:
            b = torch.cuda.FloatTensor()

        v_ij=torch.Tensor()
        a_ij=torch.Tensor()

        if GPU_available :
            v_ij=v_ij.to(device)
            a_ij=a_ij.to(device)   

        for i, v_i in enumerate(gr):
            for v_j in gr[i+1:]:
                element_wise = torch.mul(v_i,v_j)
                v_ij = torch.cat((v_ij,element_wise.reshape(-1,1)),dim=-1)
                a_ij = torch.cat((a_ij,attention(element_wise)))

        v_ij = v_ij.reshape(embedding_dim,-1)
        a_hat_ij = nn.Softmax(dim=0)(a_ij) # relative impact

        product = torch.mm(p,torch.mul(v_ij,a_hat_ij)).sum()
        a = torch.cat((a,product.reshape(1)))

    return a

In [ ]:
def Attention(embedding_dim, drop_ratio=0):
    L1 = nn.Linear(embedding_dim,30)
    L2 = nn.Linear(30,1)
    
    def forward(x):
        out = L1(x)
        out = nn.Sigmoid()(out)
        out = L2(out)
        out = nn.Sigmoid()(out)
        return out

In [ ]:
def afm(group):
    first = FirstOrder(group)
    second = AttentiveSecond(group)
    asd = nn.Sigmoid()(first+second)*2-1
    return asd

In [ ]:
def train(model, data, lr): 
    lr = lr
    optimizer = optim.Adam(model.parameters(),lr)

    total_loss = 0
    loss = 0
    for batch_id, (group, label) in enumerate(data):
        if GPU_available:
            group = group.to(device)
            label = label.to(device)

        model.zero_grad()
        output = model(group)
        loss = nn.MSELoss()(output,label)
        total_loss += loss
        loss.backward()
        optimizer.step()
        print('batch_id : %d, loss : %f' %(batch_id,loss))
    total_loss /= batch_id
    print(total_loss)

In [ ]:
trainData = Data('test')
num_user = trainData.user_number()
model = afm(num_user)

for epoch in range(epoch):
    model.train()
    print(epoch+1)
    train(model, trainData.dataLoader(), lr)